Since we had already identified

1. We would prefer likely ranges to validate the cost
3. We have identified that the principle of instance-based learning can be useful
2. We have a quick and dirty solution to categorize the diamonds by the 4C's and then predict the price

We know that there are 3 models that we can try in order to solve the problem

1. A form of polynomial regression using the logarithm as seen during exploration
2. A decision tree or a knn regression as examples of instance based learning
3. The manual solution of making the first steps of the tree by ourselves using the 4C

In [1]:
# Data management and computation
import numpy as np
import pandas as pd
import os

out_path='Databases'
# Read databases, considering that the first column is just an index
X_train = pd.read_csv(os.path.join(out_path,'X_train.csv'), index_col=0, encoding='utf8')
X_test = pd.read_csv(os.path.join(out_path,'X_test.csv'), index_col=0, encoding='utf8')
y_train = pd.read_csv(os.path.join(out_path,'y_train.csv'), index_col=0, encoding='utf8')
y_test = pd.read_csv(os.path.join(out_path,'y_test.csv'), index_col=0, encoding='utf8')
Gringotts = pd.read_csv(os.path.join(out_path,'Gringotts.csv'), index_col=0, encoding='utf8')

In [2]:
import plotly.express as px


# Manual Solution

Here we make the segmentation of the categories and then predict using the most similar diamond. This is similar to a regression tree with a 1-nearest neighbor strategy

In [3]:
# Library for scaling data
from sklearn.preprocessing import MinMaxScaler

# Eliminate complicated na
X_train = pd.concat([X_train,y_train], axis=1)
X_train.dropna(axis=0, how='any', inplace=True)
y_train = X_train['Price']
X_train = X_train.iloc[:,:-1]

In [4]:
X_train.head()

,Carat,Cut,Color,Clarity,Depth,Table,x,y,z,latitude,longitude
13451,1.08,Very Good,H,VS2,58.1,62.0,6.72,6.74,3.91,51.266667,-114.016667
6428,1.01,Fair,H,VS1,66.1,55.0,6.28,6.24,4.14,30.0472222,-99.140000
13702,1.51,Good,E,I1,57.5,59.0,7.56,7.51,4.33,49.283333,-122.850000
15247,1.07,Premium,D,SI1,63.0,57.0,6.57,6.47,4.11,44.2619444,-88.415278
36758,0.30,Ideal,E,VVS1,61.2,57.0,4.36,4.33,2.66,30.3569444,-87.163889


In [5]:
def Manual_solution(X_Train, X_Test,columns, y_Train, printmode=False):
    """Perform the manual computation of the nearest diamond to each diamond in 
    X_test in X_train (according to the columns) in the same categories of color, 
    cut and clarity.
    Returns
    -------
    Manual estimation: the price estimations
    Distances: Distance between diamonds in the feature space given by columns
    """
    Manual_Estimation = []
    Distances = []
    for i,item in enumerate(X_Test.values):
        if printmode:
            print('Analizing {} diamond'.format(item))
        # Filter information in the same category
        cut,col,clar = '"'+item[1]+'"','"'+item[2]+'"','"'+item[3]+'"'
        local_index = X_Train.query('Cut=={} and Color=={} and Clarity=={}'.format(cut,col,clar)).index
        if printmode:
            print('{} similar diamonds in train set'.format(local_index.shape[0]))
        # Scale data to be in [0,1]
        min_max_scaler = MinMaxScaler()
        X_local = X_Train.loc[local_index]
        X_local = X_local.iloc[:,columns]
        X_local_scaled = min_max_scaler.fit_transform(X_local)
        y_local = y_Train.loc[local_index]
        # Get the position of the closest diamond in this feature space
        position = ((X_local_scaled - min_max_scaler.transform(item[columns].reshape(1,-1)))**2).sum(axis=1).argmin()
        Distances.append(((X_local_scaled - min_max_scaler.transform(item[columns].reshape(1,-1)))**2).sum(axis=1).min())
        if printmode:
            print('Closest diamond {}'.format(min_max_scaler.inverse_transform(X_local_scaled[position].reshape(1,-1))))
            print('Estimated cost {}'.format(np.array(y_local)[position]))
        Manual_Estimation.append(np.array(y_local)[position])
    return (Manual_Estimation,Distances)


First we use all non-geographic variables

In [6]:
Manual_solution(X_train, Gringotts,[0,4,5,6,7,8], y_train, printmode=False)


([2394, 4860, 1614, 790, 776, 3267, 1826, 8149, 1050, 648],
 [0.00987328017656961,
  0.009960272924950553,
  0.0005224239176215657,
  0.00016456334004883176,
  0.00987191504054218,
  0.00011618827443331987,
  0.0028556084968421624,
  0.0014566007363754462,
  5.891645272296582e-05,
  0.00015158441034414876])

Now we use only the weight and the size

In [7]:
Manual_solution(X_train, Gringotts,[0,6,7,8], y_train, printmode=False)

([2428, 3880, 1614, 790, 828, 3267, 1835, 8773, 917, 648],
 [0.0003552739262928493,
  0.0002881191906547284,
  2.8596757127741943e-05,
  0.00016456334004883176,
  3.2869924717465896e-05,
  3.014892812680253e-06,
  7.217089468171754e-05,
  0.00015130229710720922,
  5.574292550591228e-05,
  0.00015158441034414876])

Now only weight

In [8]:
Manual_solution(X_train, Gringotts,[0], y_train, printmode=False)

([2283, 3880, 1746, 790, 828, 3267, 1873, 9820, 931, 648],
 [0.0, 4.271861249946571e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

This suggest that the depth and table are more or less irrelevant, and the estimation of the 0,1,3,4,5,6 and 9 diamonds is more or less stable

# Linear Model

Now we proceed to fit a lienar regression model. For me it's not clear that the ordered categorical variables are equally spaced, therefore I will only consider them as categories

In [9]:
import statsmodels.api as sm

In [10]:
# Data preparation
X_train_linear = X_train.loc[:,['Carat','Cut','Color','Clarity','Depth','Table','x','y','z']]
X_train_linear['vol'] = X_train['x']*X_train['y']*X_train['z']

X_train_linear = pd.concat([X_train_linear,y_train], axis=1)
X_train_linear.dropna(axis='index',how='any', inplace=True)


In [11]:
X_train_linear

,Carat,Cut,Color,Clarity,Depth,Table,x,y,z,vol,Price
13451,1.08,Very Good,H,VS2,58.1,62.0,6.72,6.74,3.91,177.094848,5524
6428,1.01,Fair,H,VS1,66.1,55.0,6.28,6.24,4.14,162.235008,4044
13702,1.51,Good,E,I1,57.5,59.0,7.56,7.51,4.33,245.838348,5601
15247,1.07,Premium,D,SI1,63.0,57.0,6.57,6.47,4.11,174.707469,6124
36758,0.30,Ideal,E,VVS1,61.2,57.0,4.36,4.33,2.66,50.217608,956
...,...,...,...,...,...,...,...,...,...,...,...
11284,1.00,Very Good,E,SI2,58.7,59.0,6.50,6.58,3.84,164.236800,4975
44732,0.47,Ideal,G,VVS2,62.2,53.0,5.00,5.04,3.12,78.624000,1618
38158,0.34,Ideal,G,IF,61.9,54.0,4.46,4.49,2.77,55.470358,1014
860,0.90,Premium,J,SI1,62.8,59.0,6.13,6.03,3.82,141.202098,2871


In [12]:
# Fit linear model
from statsmodels.formula.api import ols

mod = ols(formula="np.log(Price) ~ np.log(Carat)+Depth+Table+x+y+z+vol+C(Cut)+C(Color)+C(Clarity)", 
          data=X_train_linear)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(Price)   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                 9.668e+04
Date:                Tue, 21 Sep 2021   Prob (F-statistic):               0.00
Time:                        06:18:47   Log-Likelihood:                 24242.
No. Observations:               40791   AIC:                        -4.843e+04
Df Residuals:                   40766   BIC:                        -4.822e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               7.3255    

We know that volume and Carat are highly correlated, so we will begin by adding covariates

In [13]:
mod = ols(formula="np.log(Price) ~ np.log(Carat)+C(Cut)+C(Color)+C(Clarity)", 
          data=X_train_linear)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(Price)   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                 1.284e+05
Date:                Tue, 21 Sep 2021   Prob (F-statistic):               0.00
Time:                        06:18:47   Log-Likelihood:                 24164.
No. Observations:               40791   AIC:                        -4.829e+04
Df Residuals:                   40772   BIC:                        -4.813e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               7.8563    

This model has the same R squared but higher AIC, let's see its predictions and the length of predicted intervals

In [14]:
Linear = np.exp(res.predict(exog=dict(Carat=Gringotts['Carat'],Cut=Gringotts['Cut'],
            Color=Gringotts['Color'], Clarity=Gringotts['Clarity'])))

In [15]:
Linear

1     2603.822952
2     4092.148699
3     1635.284209
4      680.312135
5      664.985114
6     3198.384687
7     1922.710468
8     8249.015541
9     1185.740859
10     627.428262
dtype: float64

In [16]:
predictions = res.get_prediction(exog=dict(Carat=Gringotts['Carat'],Cut=Gringotts['Cut'],
            Color=Gringotts['Color'], Clarity=Gringotts['Clarity']))
obs_interval = predictions.summary_frame(alpha=0.05)[['obs_ci_lower','obs_ci_upper']]
mean_interval = predictions.summary_frame(alpha=0.05)[['mean_ci_lower','mean_ci_upper']]

In [17]:
np.exp(obs_interval)

,obs_ci_lower,obs_ci_upper
0,2002.802709,3385.203115
1,3147.785096,5319.829804
2,1257.908567,2125.873466
3,523.282807,884.463611
4,511.518078,864.495745
5,2460.214673,4158.037392
6,1478.980358,2499.570413
7,6345.244294,10723.977556
8,912.092832,1541.489347
9,482.622152,815.682045


In [18]:
np.exp(mean_interval)

,mean_ci_lower,mean_ci_upper
0,2585.022309,2622.760331
1,4073.912792,4110.466234
2,1628.490075,1642.106689
3,675.486890,685.171849
4,661.634806,668.352387
5,3179.877936,3216.999146
6,1912.898957,1932.572304
7,8204.757726,8293.512090
8,1179.794356,1191.717334
9,623.854595,631.022400


Let's see the standard deviation. In the log-normal distribution, the mean is multiplied by a factor of $\exp(\sigma^2/2)$

In [19]:
y_hat = np.array(res.predict(exog=dict(Carat=X_train_linear['Carat'],Cut=X_train_linear['Cut'],
            Color=X_train_linear['Color'], Clarity=X_train_linear['Clarity'])))
np.var(y_hat-np.log(np.array(y_train.values)))

0.017905312490017394

In [20]:
np.exp(np.var(y_hat-np.log(np.array(y_train.values)))/2)

1.008992851132724

## See how it works on Test set

In [21]:
y_hat = np.exp(np.array(res.predict(exog=dict(Carat=X_test['Carat'],Cut=X_test['Cut'],
            Color=X_test['Color'], Clarity=X_test['Clarity']))))

In [22]:
px.scatter(x=np.log(y_hat), y=np.log(np.array(y_test['Price'])))

In [23]:
px.scatter(x=y_hat, y=np.array(y_test['Price']))

Graphically it seems ok

In [24]:
mod = ols(formula="np.log(Price) ~ np.log(Carat)+np.log(Depth)+C(Cut)+C(Color)+C(Clarity)", 
          data=X_train_linear)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(Price)   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                 1.217e+05
Date:                Tue, 21 Sep 2021   Prob (F-statistic):               0.00
Time:                        06:18:49   Log-Likelihood:                 24165.
No. Observations:               40791   AIC:                        -4.829e+04
Df Residuals:                   40771   BIC:                        -4.812e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               7.9691    

The Depth is not significant in our model, therefore, we omit it, and we prefer our original model

I've ran out of time to implement other ideas...